In [1]:
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Load rhyme text files
def load_rhyme_texts(path="Data"):
    texts = []
    for filename in os.listdir(path):
        if filename.endswith(".txt"):
            with open(os.path.join(path, filename), "r", encoding="utf-8") as f:
                texts.append(f.read())
    return texts

# Load documents
docs = load_rhyme_texts()

# Embed using SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embedder.encode(docs).astype('float32')

# Load QA model
qa = pipeline("text2text-generation", model="google/flan-t5-base")

# Manual L1 search function
def l1_search(query_emb, doc_embeddings, top_k=3):
    dists = np.sum(np.abs(doc_embeddings - query_emb), axis=1)
    top_k_idx = np.argsort(dists)[:top_k]
    return top_k_idx

# Query
def query_rhymes(prompt, top_k=3):
    query_emb = embedder.encode([prompt]).astype('float32')[0]  # Single query vector
    top_k_idx = l1_search(query_emb, doc_embeddings, top_k=top_k)
    context = "\n".join([docs[i] for i in top_k_idx])
    full_prompt = f"Given the following nursery rhymes:\n{context}\n\nQuestion: {prompt}"
    response = qa(full_prompt, max_length=500, do_sample=False)[0]['generated_text']
    return response

# Interactive interface
if __name__ == "__main__":
    while True:
        query = input("Ask about a rhyme (or press Enter to exit): ")
        if not query.strip():
            break
        result = query_rhymes(query)
        print("Answer:", result)


C:\Users\roshn\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use cpu


Ask about a rhyme (or press Enter to exit):  Where did humpty dumpty sat


Answer: on a wall


Ask about a rhyme (or press Enter to exit):  who went up the hill


Token indices sequence length is longer than the specified maximum sequence length for this model (556 > 512). Running this sequence through the model will result in indexing errors


...

Ask about a rhyme (or press Enter to exit):  what was pocket full of


Answer: tuppenny rice


Ask about a rhyme (or press Enter to exit):  where did the mouse ran


Answer: up the clock


Ask about a rhyme (or press Enter to exit):  
